In [1]:
from sklearn.datasets import make_classification
from torch import nn, optim
from skorch import NeuralNetClassifier
import numpy  as np
# sys.path.append('./pyperch')
# from pyperch.neural.backprop_nn import BackpropModule
import matplotlib.pyplot as plt
import torch
import random
from ucimlrepo import fetch_ucirepo
from sklearn.metrics import accuracy_score, make_scorer, f1_score, confusion_matrix, log_loss

import matplotlib.pyplot as plt
import numpy as np
from sklearn.manifold import TSNE
from pipelines import KMeans_pipeline, TSNE_pipeline
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import helpers
import importlib
import pipelines
import pipeline_helpers
import seaborn as sns
from scipy.stats import kurtosis
from notebook_helper import reload_notebook_cell, generate_seed



## Get Data

In [3]:

from generate_data import get_and_preprocess_sdas_data, get_and_preprocess_raisin_data, y_label_encode, CATEGORY_COLUMNS_UCML
from sklearn.preprocessing import  LabelEncoder
reload_notebook_cell()

X_train_sdas, X_test_sdas, y_train_sdas, y_test_sdas = get_and_preprocess_sdas_data()
X_train_raisin, X_test_raisin, y_train_raisin, y_test_raisin = get_and_preprocess_raisin_data()

# Initialize and fit LabelEncoder
y_train_encoded_sdas, y_test_encoded_sdas, label_encoder_sdas = y_label_encode(y_train_sdas,y_test_sdas)
y_train_encode_raisin, y_test_encode_raisin, label_encoder_raisin = y_label_encode(y_train_raisin, y_test_raisin)

# get data to correct type for NN
y_train_encoded_sdas = y_train_encoded_sdas.astype(np.int64)
y_train_encode_raisin = y_train_encode_raisin.astype(np.int64)

c:\Users\WarrenKushner\Documents\ML\A3\generate_data.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[(X["Marital Status"]  != 1) & (X["Marital Status"] != 2) & (X["Marital Status"] != 4)]["Marital Status"] = -1


Removed columns: ["encoding__Mother's qualification_18", "encoding__Mother's qualification_44", "encoding__Father's qualification_18", "encoding__Father's occupation_161"]
Number of features after removal: 200


In [20]:
# import Kfold
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.neighbors import KNeighborsClassifier
reload_notebook_cell()
cv = KFold(n_splits=4, shuffle=False)
scoring = make_scorer(f1_score, average='macro')
best_net_inputs =   {
    "module__hidden_units": 80,
    "module__hidden_layers": 2,
    "module__activation": torch.nn.ReLU(),
    "module__input_dim": 204,
    "optimizer__lr": 0.002000,
    "max_epochs": 25,
    "module__input_dim": 200,
    "module__output_dim": 3,

}
net_pipeline_best = pipelines.NN_pipeline(X_train_sdas,CATEGORY_COLUMNS_UCML,  **best_net_inputs)
# scores_nn = cross_validate(net_pipeline_best, X_train_sdas, y_train_encoded_sdas, cv=cv,
#                         scoring=('f1_macro'),
#                         return_train_score=True)
net_pipeline_best.fit(X_train_sdas, y_train_encoded_sdas.astype(np.int64))


cpu


array([0, 2, 2, 1, 0, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0,
       0, 2, 2, 2, 0, 2, 1, 0, 1, 0, 1, 0, 0, 2, 2, 1, 2, 2, 0, 0, 1, 0,
       0, 2, 2, 2, 2, 2, 1, 2, 2, 0, 1, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 1, 2, 2,
       2, 2, 0, 2, 2, 2, 0, 0, 0, 1, 0, 0, 2, 2, 1, 0, 0, 1, 2, 0, 2, 0,
       0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 1, 0, 2, 0, 2, 2, 2, 2,
       2, 0, 2, 2, 0, 2, 0, 1, 1, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 0, 2,
       0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 1, 1, 0, 1, 2, 0, 2, 2, 2, 2, 0, 2,
       1, 0, 1, 0, 0, 2, 0, 0, 2, 0, 0, 2, 2, 0, 2, 1, 0, 1, 1, 2, 2, 2,
       0, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 0, 2, 2, 2, 0, 2, 0, 0, 1,
       2, 2, 0, 2, 0, 0, 2, 0, 0, 2, 2, 0, 1, 0, 1, 0, 0, 2, 2, 2, 2, 0,
       1, 1, 2, 0, 0, 0, 2, 1, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 0,
       2, 2, 2, 2, 2, 0, 0, 2, 0, 0, 2, 2, 1, 0, 2, 2, 2, 2, 1, 0, 0, 1,
       0, 1, 1, 0, 0, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2,